# Final Report
<h1> New York City Property Sales </h1>



## Team Members

* **Aviv Farag** - af3228@drexel.edu

* **Jospeh Logan** - jrl362@drexel.edu

* **Abdulaziz Alquzi** - aa4472@drexel.edu


---
## Discussion 

We are data science consultants who are contracted by property management investors in New York City. Their company, supported by investors, wants to buy residential real estate in NYC at the cheapest price possible, renovate, then resell within a year. The renovation analysis is outside the scope of this project, but they want a baseline model that can predict the price of residential real-estate in order to :
 
1. Identify potential undervalued listed properties to buy
2. Predict market price when it’s time to sell in order to sell quickly while maximizing return on investment
 
Because the want to renovate and sell the properties quickly, they want less than 10 residential units, and properties less than 5 million each but are at least ten thousand.


---
## Exploratory Data Analysis 

### Columns Description 

1. **Borough** is the borough in which the unit is located: '1':'Manhattan', '2':'Bronx', '3': 'Brooklyn', '4':'Queens','5':'Staten Island'. Location is a key feature in real estate, and this is especially true in NYC. For the purposes of exploratory analysis, we have converted the numeric values into their proper names.  Below you will see a clear distinction in price by neighborhood, with Manhattan being much more expensive. For constructing the model we will likely one hot encode.
 
1. **Neighborhood** is the specific neighborhood within the borough. There is a strong relationship with neighborhood and sale price. Much like borough, because the neighborhoods are not explicitly ranked, a one hot coding strategy will likely be used
 
1. **Building Class Category** is an important feature as it separates between attached and unattached houses or elevator apartments
 
1. **Tax class a present** is the tax class and is heavily correlated with both sale price and tax class at time of sale. Tax class and number of units as correlated as the tax class depends on how may units. There is a big risk of data leakage with this feature. The models real world success would depend on accurately determining the tax class before selling or purchasing a property. Because of this, we may want to remove this feature.
[Source](https://blocksandlots.com/property-taxes-in-nyc-all-you-need-to-know/)

1. **Block and Lot** The combination of borough, block, and lot forms a unique key for property in New York City. Commonly called a BBL
[Source](https://blocksandlots.com/property-taxes-in-nyc-all-you-need-to-know/)
 
1. **Building Class at present** <br><br>
According to [nyc.gov](https://www1.nyc.gov/assets/finance/downloads/pdf/07pdf/glossary_rsf071607.pdf): <br>
“The Building Classification is used to describe a property’s constructive use. The first position of the Building Class is a letter that is used to describe a general class of properties (for example “A” signifies one-family homes, “O” signifies office buildings. “R” signifies condominiums). The second position, a number, adds more specific information about the property’s use or construction style (using our previous examples “A0” is a Cape Cod style one family home, “O4” is a tower type office building and “R5” is a commercial condominium unit). The term Building Class used by the Department of Finance is interchangeable with the term Building Code used by the Department of Buildings.”
<br>
Because this feature has direct overlap with many other features, we will likely remove it.

1. **Address** Is the actual address. Because of the variance we will remove this feature. However, it could be potentially used to crosswalk longitude and latitude, but that would require an additional dataset.
 
1. **Zip Code** Zip codes are difficult to work with in machine learning problems because it’s an integer and a higher or lower zip code won’t necessarily mean it’s better or worse. If we are going to use it we will one hot encode
 
1. **Residential Units** are the number of residential units for sale. This is correlated with price as 8 units will likely cost more than 2 in a similar neighborhood. For exploratory analysis we examine the price per unit.
 
1. **Land Square Feet** is the land area of the property. This is a valuable feature for predicting price
 
1. **Gross Square Feet** According to nyc.gov: “The total area of all the floors of a building as measured from the exterior surfaces of the outside walls of the building, including the land area and space within any building or structure on the property.”  This is also a valuable feature for predicting price. However, it’s important for compare between this and location as a smaller property in the center of Manhattan, may be more expensive than a much larger property in Staten Island.
 
1. **Year Built** is the year the structure was built. Many of the properties were built a long time ago, but it’s worth further testing this feature before elimination. 
Tax class at time of sale: See above for tax class. It will be difficult to accurately predict this and it has a very high risk of data leakage. This feature will almost certainly be removed.

1. **Tax Class at Time of Sale** - <br> 
  &nbsp;[Property Tax Rate](https://www1.nyc.gov/site/finance/taxes/property-tax-rates.page "Click here to move to NYC property tax rates") for 2016/2017 was:
    - Class 1 - 19.991\%	
    - Class 2 - 12.892\%	
    - Class 3 - 10.934\%	
    - Class 4 - 10.574\% <br><br>
        **Note**:This feature might cause data leakage due to its correlation with SALE PRICE (higher price -> higher tax), and therefore we will not use it in our machine learning algorithm.

1. **Sale Price** is our target variable. Due to the scope of the business problem we are limiting the dataset to 10,000 and 5,000,0000
 
1. **Sale Date** Is the date of the sale. We may want to look at the sale month to determine if we can purchase a property in a slower month for real estate i.e. buy in the winter cheaply and resell in a hotter month like this the spring.

### Residential Units vs. Sale Price

We used Seaborn package to plot a boxplot of residential units versus sale price. 

```
plt.figure(figsize=(10,6))
sns.boxplot(x='RESIDENTIAL UNITS', y='SALE PRICE', data=df)
plt.title('Residential Units vs Sale Price')
plt.show()
```

<center><img  src="./img/res_vs_saleprice.png" width = "60%" />
<br>
<b><u>Fig. 1</u></b>: Box plot of Residential Units and Sale Price </center>

The mean SALE PRICE of 8 residential units is higher than the mean value of any other number of residential units. Also, mean SALE PRICE increases as the residential units increase up to 8 residential units. Then, it goes down for 9 residential units. Moreover, there are a lot of outliers in 1,2,3, and 4 residential units which could be due to Borough or other parameters that were not taken into account in this plot.

### Borough vs. Sale Price

```
plt.figure(figsize=(10,6))
sns.boxplot(x='BOROUGH', y='SALE PRICE', data=df)
plt.title('Borough vs Sale Price')
plt.show()
```


<center><img  src="./img/borough_vs_saleprice.png" width = "60%" />
<br>
<b><u>Fig. 2</u></b>: Borough vs Sale Price </center>

The figure above presents the distribution of Sale price among Borough. The mean SALE PRICE in Manhattan is the highest. Also, the distribution of SALE PRICE in Manhattan is the longest (up to ~4.5 million without outliers).

### Correlation Table

```
# New copy of dataframe
df_copy = df.copy()

# Define costum label encoder (inline lambda function)
label_encoder = lambda x: {uniq : i for i,uniq in enumerate(df[x].unique().tolist())}

df_copy["SALE PRICE"] = np.log(df_copy["SALE PRICE"])

# Map different columns to numbers
df_copy['NEIGHBORHOOD'] = df_copy['NEIGHBORHOOD'].map(label_encoder('NEIGHBORHOOD'))
df_copy['BUILDING CLASS CATEGORY'] = df_copy['BUILDING CLASS CATEGORY'].map(label_encoder('BUILDING CLASS CATEGORY'))
df_copy["TAX CLASS AT PRESENT"] = df_copy["TAX CLASS AT PRESENT"].map({'1': 1,'1A':1, '1B':1, '1C':1, 
                                                             '2':2, '2C':2, '2B':2, '2A':2,
                                                             '4':4})
```
We made a copy of the DataFrame and defined a costumed label encoder using lambda function. Additionally, we used logarithm on the target variable due to large amount of outliers. Then, we converted string values to numbers for some features using our label encoder function. The correlation results are presented in the figure below:


<center><img  src="./img/correlation_table.png" width = "80%" />
<br>
<b><u>Fig. 3</u></b>: Correlation table </center>


Based on this correlation table and the other figures above we chose a few features for our machine learning model:
1. Borough -> values 1 to 5
2. Neighborhood -> using label encoder to transform strings to numbers
3. Block -> numerical 
4. Lot -> numerical
5. Building Class Category -> Strings (OneHotEncoder because number of attributes is low)
6. Residential Units -> values 1 to 9
7. Gross Square Feet -> numerical



---
## Machine Learning 

### Models
We implemented Linear Regression, Decision Tree Regressor, Extre Tree Regressor and Random Forest Regressor in order to predict property price. 

### Data Preparation



#### Label Encoder Function

Defined label encoder function to ease the process of converting unique values (strings) to numbers:
```
# Define costum label encoder (inline lambda function)
label_encoder = lambda x: {uniq : i for i,uniq in enumerate(df[x].unique().tolist())}

def label_encode(column = ""):
    if column:
        df[column] = df[column].map(label_encoder(column))
```



#### Tree Regressors



```
# Label Encoder for neighborhood and Building class categroy
label_encode("NEIGHBORHOOD")
label_encode("BUILDING CLASS CATEGORY")

# DataFrame for Linear Regression (remove outliers)
lr_df = df.copy()

# Log sale price
df["SALE PRICE"] = np.log(df["SALE PRICE"])

# Chosen features for ML models
modified_df = df[["BOROUGH",
                  "NEIGHBORHOOD",
                  "BLOCK",
                  "LOT",
                  "BUILDING CLASS CATEGORY",
                  "RESIDENTIAL UNITS",
                  "GROSS SQUARE FEET",
                  ]]

# Target column
y = df["SALE PRICE"]

# Split to train and test
tr_x_train, tr_x_test, tr_y_train, tr_y_test = train_test_split(modified_df,y, test_size=0.2, random_state=42)

```



#### Linear Regression
In order to deal with outliers in an efficient way and without removing too many instances from our dataset, we trained the linear regression model on price per unit in a logarithm scale. In other words, we divided the SALE PRICE column with RESIDENTIAL UNITS and the results we transformed to a logarithm scale. In the validation section, we would like to convert the predictions back to the original SALE PRICE's domain, and therefore we initalized two variables: lr_y_test and lr_y_test_ru. The first one (lr_y_test) is SALE PRICE which we would like to predict and the second one (lr_y_test_ru) is the residential units of the target column which we need in order to convert price per unit back to SALE PRICE. 



```
# Handling outliers for Linear Regression
# Calculate price per unit and transform to log
lr_df["price per unit"] = np.log(lr_df["SALE PRICE"] / lr_df["RESIDENTIAL UNITS"])

# Remove outliers 
lr_df = lr_df[(lr_df["price per unit"] > 11.7) & (lr_df["price per unit"] < 14.8)]

lr_modified = lr_df[["BOROUGH",
                  "NEIGHBORHOOD",
                  "BLOCK",
                  "LOT",
                  "BUILDING CLASS CATEGORY",
                  "RESIDENTIAL UNITS",
                  "GROSS SQUARE FEET",
                  "price per unit",
                  "SALE PRICE"]]

# Split to train and test
train_set, test_set = train_test_split(lr_modified, test_size=0.2, random_state=42)

# Split data and target in training set
lr_y_train_ppu = train_set["price per unit"].copy()
lr_x_train_ppu = train_set.drop(["SALE PRICE","price per unit","RESIDENTIAL UNITS"], axis = 1)

# Split data and target in test set
lr_y_test_ppu = test_set["price per unit"].copy()
lr_y_test = test_set["SALE PRICE"].copy()
lr_y_test_ru = test_set["RESIDENTIAL UNITS"]
lr_x_test_ppu = test_set.drop(["SALE PRICE","price per unit","RESIDENTIAL UNITS"], axis = 1)
```



### Baseline - Dummy Regressor
There are two dummy regressors, one for Linear Regression and the other one for the other regressors. The reason for having two baselines is that the model for linear regression is different because Linear Regression is sensitive to outliers, and therefore the model is different.

#### Trees Baseline 
```
print("Trees df")

# Initialize Dummy estimator 
dummy_regr = DummyRegressor(strategy="median")

# Train estimator
dummy_regr.fit(tr_x_train, tr_y_train)

# Predict 
preds = np.exp(dummy_regr.predict(tr_x_test))


# Calculate RMSE
mse = mean_squared_error(np.exp(tr_y_test), preds)
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(np.exp(tr_y_test), preds)

# Print results
print("R-Squared: ", r2_score(np.exp(tr_y_test), preds))
print("Root Mean Square Error: ", rmse)
print("Mean Absolute Error: ", mae)
print()
```
results:


```
Trees df
R-Squared: -0.107
Root Mean Square Error: 602,116.561
Mean Absolute Error: 346,494.955
```


#### Linear Regression Baseline
As mentioned earlier, we used a different model for Linear Regression due to it sensitivity to outliers. 
```
print("Linear Regression df")

# Initialize Dummy estimator 
lr_dummy_regr = DummyRegressor(strategy="median")

# Train estimator
lr_dummy_regr.fit(lr_x_train_ppu, lr_y_train_ppu)

# Predict 
lr_preds = np.exp(lr_dummy_regr.predict(lr_x_test_ppu))

# Calculate RMSE
mse = mean_squared_error(np.exp(lr_y_test_ppu) * lr_y_test_ru, lr_preds * lr_y_test_ru)
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(np.exp(lr_y_test_ppu)* lr_y_test_ru, lr_preds * lr_y_test_ru)

# Print results
print("R-Squared: ", r2_score(np.exp(lr_y_test_ppu) * lr_y_test_ru, lr_preds * lr_y_test_ru))
print("Root Mean Square Error: ", rmse)
print("Mean Absolute Error: ", mae)
print()
```
results:


```
Linear Regression df
R-Squared: -0.695
Root Mean Square Error: 680,709.220
Mean Absolute Error: 475,646.653
```



### Pipeline

```
numeric_features = ['GROSS SQUARE FEET',"BLOCK","LOT", "NEIGHBORHOOD"]
numeric_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())])


ohe_features = ["BUILDING CLASS CATEGORY", "BOROUGH"]
ohe_transformer = OneHotEncoder(sparse = False, handle_unknown = 'ignore')


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('ohe', ohe_transformer, ohe_features),
        ])

# Linear regression Estimator
lr = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regression', LinearRegression())])

# Decision Tree Regressor
dtr = Pipeline(steps=[
                      ('dtr', DecisionTreeRegressor())])

# Extra Tree Regressor
etr = Pipeline(steps=[
                      ('etr', BaggingRegressor(ExtraTreeRegressor(random_state=42),
                                                        random_state = 42))])
# Random Forest Regressor
rfrr = Pipeline(steps=[
                      ('rfregression', RandomForestRegressor(random_state = 42, n_jobs= -1, 
                                                             n_estimators = 50, 
                                                             min_samples_leaf = 5))])
```

We scaled numeric features such as Gross Square Feet, Block, Lot and Neighborhood. We used OneHotEncoder for features with a few unique values such as Building class category and Borough. Additionally, we didn't use the preprocessing step in the pipeline of Random Forest, Decision Tree Regressor and Extra Tree Regressor since data preparation is not required for those models.



### Cross Validation


We implemented two algorithms for cross validation. One uses the sklearn method called cross_validate, and the other one is a modified version of a code taken from Jupyter Notebook Week 2. The former was used for Decision Tree Regressor, Extra Tree Regressor and Random Forest, while the latter was utilized for linear regression due to the process of target engineering in this model. 


#### Tree Regressors


Here is the function we created using the method cross_validate:
```
def validation(models = [], estimators = [], training = [], cv = 5, train_score = False):
    if len(models) != len(estimators):
        print("Error: model names and estimator must have the same length")
        return

    for model, estimator in zip(models, estimators):
        scores = cross_validate(estimator, training[0], training[1], cv=5,
                            scoring=('r2',
                                        'neg_mean_squared_error',
                                        'neg_mean_absolute_error'),
                            return_train_score=False)
        print(model)
        print("R-Squared: {:,.3f}".format(np.mean(scores["test_r2"])))
        print("Root Mean Squared Error: {:,.3f}".format(np.mean(np.sqrt(-scores["test_neg_mean_squared_error"]))))
        print("Mean Absolute Error: {:,.3f}".format(np.mean(-scores["test_neg_mean_absolute_error"])))
        print()


validation(models = ["Decision Tree Regressor", "Extra Tree Regressor", "Random Forest"], 
           estimators = [dtr, etr, rfrr],
           training = [tr_x_train, np.exp(tr_y_train)])
```

Results:


```
Decision Tree Regressor
R-Squared: 0.388
Root Mean Squared Error: 468,186.233
Mean Absolute Error: 263,593.068

Extra Tree Regressor
R-Squared: 0.634
Root Mean Squared Error: 362,126.920
Mean Absolute Error: 211,437.049

Random Forest
R-Squared: 0.649
Root Mean Squared Error: 354,855.099
Mean Absolute Error: 206,175.793
```

Cross Validation results are much better than our baseline. 


#### Linear Regression


Below is the modified code taken from Jupyter Notebook Week 2:
```
# initialize the model
kf = KFold(n_splits=10, random_state = 42, shuffle = True)

# make a list to store our RMSE 
lr_RMSEs, lr_R2, lr_MAE = [], [], []

# loop over the k folds
for train_index, validate_index in kf.split(lr_x_train_ppu):
    
    # train the model using the training set
    lr.fit(lr_x_train_ppu.iloc[train_index,], lr_y_train_ppu.iloc[train_index,])
    
    # predict on a the validation set
    predictions = lr.predict(lr_x_train_ppu.iloc[validate_index,])
    predictions = np.exp(predictions) * lr_y_train_ru.iloc[validate_index,]

    test_set = lr_y_train.iloc[validate_index]

    # Append scores
    lr_R2.append(r2_score(test_set, predictions))
    
    mse = mean_squared_error(test_set,predictions)
    lr_RMSEs.append(np.sqrt(mse))
    
    lr_MAE.append(mean_absolute_error(test_set, predictions))
    

    
# let's look at the output from k fold
print("Linear Regression: ")
print("R-Squared: {:,.3f}".format(np.mean(lr_R2)))
print("Root Mean Squared Error: {:,.3f}".format(np.mean(lr_RMSEs)))
print("Mean Absolute Error: {:,.3f}".format(np.mean(lr_MAE)))
print()
```
Results:


```
Linear Regression: 
R-Squared: 0.252
Root Mean Squared Error: 445,688.977
Mean Absolute Error: 262,219.829
```
The results are better than the baseline, but the error is too big to rely on such model. There is no profit margin if the error is 262,000+. 




### Testing
After training the different models, we got the results:


```
Descision Tree:
R-Squared: 0.401
Root Mean Square Error: 442,844.551
Mean Absolute Error: 249,737.346

Extra-Tree Regressor:
R-Squared: 0.652
Root Mean Square Error: 337,775.405
Mean Absolute Error: 194,745.277

Random Forest:
R-Squared: 0.650
Root Mean Square Error: 338,407.994
Mean Absolute Error: 190,655.493

Linear Regression: 
R-Squared: 0.327
Root Mean Square Error: 428,974.407
Mean Absolute Error: 259,014.542
```

Decision Tree and Linear Regression are not preforming well for this problem. Linear Regression is mainly affected by outliers, and therefore we removed them in the data cleaning process. Decision Tree is usually performing bad due to overfitting the training set. Therefore, we also tried Random Forest Regression and Extra Tree Regressor to add randomness to the model and the results are better. They would probably be much better if we had other features such as the number of rooms/bathrooms and/or number of parking lots. 

### Hyperparameter Tuning 

#### Decision Tree Regressor
RandomizedSearchCV was utilized in order to achieve best results for Decision Tree Regresson. In the code below the parameters are listed:

```
grid_param = [{"dtr__max_depth": randint(low=20, high=60),
               "dtr__min_samples_split":randint(low=2, high=6),
               "dtr__min_samples_leaf": randint(low = 2, high = 10)}]
```
##### Results


```
Decision Tree Random Search CV
R-Squared: 0.583
Root Mean Square Error: 369,410.920
Mean Absolute Error: 212,902.102

Pipeline(steps=[('dtr',
                 DecisionTreeRegressor(criterion='mse', max_depth=35,
                                       min_samples_leaf=9,
                                       min_samples_split=5))])
```
Results are much better than the previous one. Score was increased from 0.4 to 0.58. We also got a great decrease in RMSE and MAE which is great for our mission.


#### Extra Tree Regressor

RandomizedSearchCV Parameters:
```
grid_param = [{"etr__n_estimators": randint(low=20, high=60),
               "etr__max_features":randint(low=1, high=6)}]
```
##### Results


```
R-Squared: 0.673
Root Mean Square Error: 327,489.884
Mean Absolute Error: 186,690.905

Pipeline(steps=[('etr',
                 BaggingRegressor(base_estimator=ExtraTreeRegressor(criterion='mse',
                                                                    random_state=42),
                                  max_features=5, n_estimators=40,
                                  random_state=42))])
```
Slightly improved, but not significant. 



#### Random Forest
Used RandomizedSearchCV with the below parameters:
```
## Random Search CV
grid_param = [{"rfregression__n_estimators": randint(low=20, high=100),
               "rfregression__min_samples_leaf":randint(low=2, high=30)}]

```
##### Results


```
Random Forest Random Search CV
R-Squared: 0.660
Root Mean Square Error: 333,608.167
Mean Absolute Error: 187,708.521

Pipeline(steps=[('rfregression',
                 RandomForestRegressor(criterion='mse', min_samples_leaf=3,
                                       n_estimators=75, n_jobs=-1,
                                       random_state=42))])
```
Not much improved than the previous one. 




---
## Conclusions 

Our dataset mainly contains information about properties location such as Borough, Block, and Lot. It also contains bulding class and tax class. However, there is not information about the properties such as wether its renovated, number of rooms, parking lot, balcony and so on. Those added features could enhance the performance of our models, especially Random Forest and Linear Regression. 


---
## Future Considerations
To enhance our model performance there are other features that might be helpful and are missing in our dataset such as the number of rooms and the number of baths in each property.